In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [ ]:
# Carregar os dados

dados_completos = pd.read_csv('C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\DE\Data\Modelo 2\dados_completos - Copy.csv')

# Preparar as características e o alvo
colunas_relevantes = ['customer_state', 'customer_city', 'seller_state','distance', 'order_id']

# Selecionar as colunas relevantes
dados_selecionados = dados_completos[colunas_relevantes]

# Converter colunas categóricas para numéricas com redução de dummy variables
dados_selecionados = pd.get_dummies(dados_selecionados, drop_first=True)

# Selecionar a variável alvo
y = dados_completos['delivery_time']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dados_selecionados, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo
modelo = RandomForestRegressor(n_estimators=20, random_state=42)
modelo.fit(X_train, y_train)

In [14]:
# Passo 3: Previsões e Submissão
def mapeamento(dataframe, idtype, variavel):
    id = dataframe.set_index(idtype)[variavel].to_dict()
    dados_teste[variavel] = dados_teste[idtype].map(id)
# Carregar os dados de teste
dados_teste = pd.read_csv('C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\DE\Data\test.csv')

seller_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\sellersAws.csv')
product_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\productinfAws.csv')
orders_time_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\orderstimeAws.csv')
order_items_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\order_itemsAws.csv')
customer_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\customerAws.csv')
payment_data = pd.read_csv('C:\\Users\\paulo\\Downloads\\Mac2166\\paymentAws.csv')

In [15]:
dados_completos = pd.read_csv('C:\\Users\\paulo\\Downloads\\dados_completos.csv')

In [16]:
# Mapear dados adicionais
mapeamento(order_items_data, 'order_id', 'seller_id')
mapeamento(seller_data, 'seller_id', 'seller_city')
mapeamento(seller_data, 'seller_id', 'seller_state')

mapeamento(order_items_data, 'order_id', 'product_id')
mapeamento(product_data, 'product_id', 'product_category_name')
mapeamento(product_data, 'product_id', 'product_weight_g')

mapeamento(customer_data, 'customer_id', 'customer_city')
mapeamento(customer_data, 'customer_id', 'customer_state')

mapeamento(dados_completos, 'order_id', 'distance')

In [ ]:
# Selecionar as colunas relevantes no conjunto de teste
dados_teste_selecionados = dados_teste[['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at','customer_state', 'customer_city', 'seller_state', 'seller_city','distance']]

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste_selecionados = pd.get_dummies(dados_teste_selecionados, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste_selecionados = dados_teste_selecionados.reindex(columns=X_train.columns, fill_value=0)

# Fazer previsões
previsoes = modelo.predict(dados_teste_selecionados)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = 'C:/Users/paulo/Downloads/Mac2166/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")

In [ ]:
# Adicionar dados mapeados ao conjunto de teste
dados_teste = pd.merge(dados_teste, customer_data[['customer_id', 'customer_state', 'customer_city']], on='customer_id', how='left')
dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], left_on='seller_id', right_on='seller_id', how='left')
dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')

# Selecionar as colunas relevantes no conjunto de teste
colunas_relevantes_teste = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at',
                            'seller_id', 'product_id', 'customer_state', 'customer_city',
                            'seller_state', 'seller_city', 'product_category_name', 'product_weight_g']
dados_teste = dados_teste[colunas_relevantes_teste]

# Tratar valores NaN no conjunto de teste
imputer = SimpleImputer(strategy='mean')
dados_teste['customer_state'].fillna(method='ffill', inplace=True)  # Preencher NaN com valores anteriores
dados_teste['customer_city'].fillna(method='ffill', inplace=True)
dados_teste['seller_state'].fillna(method='ffill', inplace=True)
dados_teste['seller_city'].fillna(method='ffill', inplace=True)
dados_teste['product_category_name'].fillna(method='ffill', inplace=True)
dados_teste['product_weight_g'].fillna(method='ffill', inplace=True)

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste = pd.get_dummies(dados_teste, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste = pd.DataFrame(dados_teste, columns=X_train.columns)

# Tratamento dos valores NaN no conjunto de teste com o mesmo imputer do treino
dados_teste = imputer.transform(dados_teste)

# Fazer previsões com o modelo já treinado
modelo = RandomForestRegressor(n_estimators=50, random_state=42)  # Substitua pelo seu modelo treinado
previsoes = modelo.predict(dados_teste)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = '/mnt/data/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")

In [ ]:


# Mapear dados adicionais
seller_id_to_city = mapeamento(seller_data, 'seller_id', 'seller_city')
seller_id_to_state = mapeamento(seller_data, 'seller_id', 'seller_state')
product_id_to_category = mapeamento(product_data, 'product_id', 'product_category_name')
product_id_to_weight = mapeamento(product_data, 'product_id', 'product_weight_g')
customer_id_to_city = mapeamento(customer_data, 'customer_id', 'customer_city')
customer_id_to_state = mapeamento(customer_data, 'customer_id', 'customer_state')

# Adicionar dados mapeados ao conjunto de teste
dados_teste = pd.merge(dados_teste, customer_data[['customer_id', 'customer_state', 'customer_city']], on='customer_id', how='left')
dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], on='seller_id', how='left')
dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')

# Selecionar as colunas relevantes no conjunto de teste
colunas_relevantes_teste = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at',
                            'seller_id', 'product_id', 'customer_state', 'customer_city',
                            'seller_state', 'seller_city', 'product_category_name', 'product_weight_g']
dados_teste = dados_teste[colunas_relevantes_teste]

# Tratar valores NaN no conjunto de teste
imputer = SimpleImputer(strategy='mean')
dados_teste['customer_state'].fillna(method='ffill', inplace=True)  # Preencher NaN com valores anteriores
dados_teste['customer_city'].fillna(method='ffill', inplace=True)
dados_teste['seller_state'].fillna(method='ffill', inplace=True)
dados_teste['seller_city'].fillna(method='ffill', inplace=True)
dados_teste['product_category_name'].fillna(method='ffill', inplace=True)
dados_teste['product_weight_g'].fillna(method='ffill', inplace=True)

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste = pd.get_dummies(dados_teste, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste = pd.DataFrame(dados_teste, columns=X_train.columns)

# Tratamento dos valores NaN no conjunto de teste com o mesmo imputer do treino
dados_teste = imputer.transform(dados_teste)

# Fazer previsões com o modelo já treinado
modelo = RandomForestRegressor(n_estimators=50, random_state=42)  # Substitua pelo seu modelo treinado
previsoes = modelo.predict(dados_teste)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = '/mnt/data/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")


In [ ]:
dados_teste = pd.merge(dados_teste, customer_data[['customer_id', 'customer_state', 'customer_city']], on='customer_id', how='left')

# Verificar se a coluna 'seller_id' está presente no conjunto de teste antes de mesclar
if 'seller_id' not in dados_teste.columns:
    dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], left_on='seller_id', right_on='seller_id', how='left')

dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')


In [ ]:
# Adicionar dados mapeados ao conjunto de teste
dados_teste = pd.merge(dados_teste, customer_data[['customer_id', 'customer_state', 'customer_city']], on='customer_id', how='left')
dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], on='seller_id', how='left')
dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')
if 'seller_id' not in dados_teste.columns:
    dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], left_on='seller_id', right_on='seller_id', how='left')

dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')

# Tratar valores NaN no conjunto de teste
imputer = SimpleImputer(strategy='mean')
dados_teste['customer_state'].fillna(method='ffill', inplace=True)  # Preencher NaN com valores anteriores
dados_teste['customer_city'].fillna(method='ffill', inplace=True)
dados_teste['seller_state'].fillna(method='ffill', inplace=True)
dados_teste['seller_city'].fillna(method='ffill', inplace=True)
dados_teste['product_category_name'].fillna(method='ffill', inplace=True)
dados_teste['product_weight_g'].fillna(method='ffill', inplace=True)

# Selecionar as colunas relevantes no conjunto de teste
colunas_relevantes_teste = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at',
                            'seller_id', 'product_id', 'customer_state', 'customer_city',
                            'seller_state', 'seller_city', 'product_category_name', 'product_weight_g']
dados_teste = dados_teste[colunas_relevantes_teste]

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste = pd.get_dummies(dados_teste, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste = pd.DataFrame(dados_teste, columns=X_train.columns)

# Tratamento dos valores NaN no conjunto de teste com o mesmo imputer do treino
dados_teste = imputer.transform(dados_teste)

# Fazer previsões com o modelo já treinado
modelo = RandomForestRegressor(n_estimators=50, random_state=42)  # Substitua pelo seu modelo treinado
previsoes = modelo.predict(dados_teste)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = '/mnt/data/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")

In [ ]:




# Adicionar dados mapeados ao conjunto de teste
dados_teste = pd.merge(dados_teste, customer_data[['customer_id', 'customer_state', 'customer_city']], on='customer_id', how='left')
dados_teste = pd.merge(dados_teste, seller_data[['seller_id', 'seller_state', 'seller_city']], on='seller_id', how='left')
dados_teste = pd.merge(dados_teste, product_data[['product_id', 'product_category_name', 'product_weight_g']], on='product_id', how='left')

# Tratar valores NaN no conjunto de teste
imputer = SimpleImputer(strategy='mean')
dados_teste['customer_state'].fillna(method='ffill', inplace=True)  # Preencher NaN com valores anteriores
dados_teste['customer_city'].fillna(method='ffill', inplace=True)
dados_teste['seller_state'].fillna(method='ffill', inplace=True)
dados_teste['seller_city'].fillna(method='ffill', inplace=True)
dados_teste['product_category_name'].fillna(method='ffill', inplace=True)
dados_teste['product_weight_g'].fillna(method='ffill', inplace=True)

# Selecionar as colunas relevantes no conjunto de teste
colunas_relevantes_teste = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at',
                            'seller_id', 'product_id', 'customer_state', 'customer_city',
                            'seller_state', 'seller_city', 'product_category_name', 'product_weight_g']
dados_teste = dados_teste[colunas_relevantes_teste]

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste = pd.get_dummies(dados_teste, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste = pd.DataFrame(dados_teste, columns=X_train.columns)

# Tratamento dos valores NaN no conjunto de teste com o mesmo imputer do treino
dados_teste = imputer.transform(dados_teste)

# Fazer previsões com o modelo já treinado
modelo = RandomForestRegressor(n_estimators=50, random_state=42)  # Substitua pelo seu modelo treinado
previsoes = modelo.predict(dados_teste)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = '/mnt/data/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")


In [20]:
#Mapeamento 
seller_id = mapeamento(order_items_data, 'order_id', 'seller_id')
payment_tipe = mapeamento(payment_data, 'order_id', 'payment_type')
seller_id_to_city = mapeamento(seller_data, 'seller_id', 'seller_city')
seller_id_to_state = mapeamento(seller_data, 'seller_id', 'seller_state')

In [ ]:
dados_teste['customer_state'] = dados_teste['customer_id'].map(customer_id_to_state)
dados_teste['customer_city'] = dados_teste['customer_id'].map(customer_id_to_city)
dados_teste['seller_state'] = dados_teste['seller_id'].map(seller_id_to_state)
dados_teste['seller_city'] = dados_teste['seller_id'].map(seller_id_to_city)
dados_teste['product_category_name'] = dados_teste['product_id'].map(product_id_to_category)
dados_teste['product_weight_g'] = dados_teste['product_id'].map(product_id_to_weight)
dados_teste['volume'] = dados_teste['product_id'].map(product_id_to_volume)

In [7]:






customer_id_to_state = mapeamento(customer_data, 'customer_id', 'customer_state')
customer_id_to_city = mapeamento(customer_data, 'customer_id', 'customer_city')
customer_id_to_zipcode = mapeamento(customer_data, 'customer_id', 'customer_zip_code_prefix')

seller_id_to_city = mapeamento(seller_data, 'seller_id', 'seller_city')
seller_id_to_zipcode = mapeamento(seller_data, 'seller_id', 'seller_zip_code_prefix')

product_id_to_category = mapeamento(product_data, 'product_id', 'product_category_name')
product_id_to_weight = mapeamento(product_data, 'product_id', 'product_weight_g')
product_id_to_volume = mapeamento(product_data, 'product_id', 'volume')

orders_time_approve = mapeamento(orders_time_data, 'order_id', 'order_approved_at')
orders_time_purchase = mapeamento(orders_time_data, 'order_id', 'order_purchase_timestamp')


# Adicionar dados mapeados ao conjunto de teste
dados_teste['customer_state'] = dados_teste['customer_id'].map(customer_id_to_state)
dados_teste['customer_city'] = dados_teste['customer_id'].map(customer_id_to_city)
dados_teste['seller_state'] = dados_teste['seller_id'].map(seller_id_to_state)
dados_teste['seller_city'] = dados_teste['seller_id'].map(seller_id_to_city)
dados_teste['product_category_name'] = dados_teste['product_id'].map(product_id_to_category)
dados_teste['product_weight_g'] = dados_teste['product_id'].map(product_id_to_weight)
dados_teste['volume'] = dados_teste['product_id'].map(product_id_to_volume)


*******
# Selecionar as colunas relevantes no conjunto de teste
dados_teste_selecionados = dados_teste[['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at']]

# Convertendo colunas categóricas para numéricas no conjunto de teste
dados_teste_selecionados = pd.get_dummies(dados_teste_selecionados, drop_first=True)

# Garantir que as colunas do conjunto de teste são as mesmas do conjunto de treino
dados_teste_selecionados = dados_teste_selecionados.reindex(columns=X_train.columns, fill_value=0)

# Fazer previsões
previsoes = modelo.predict(dados_teste_selecionados)

# Criar o DataFrame de submissão
submissao = pd.DataFrame({
    'order_id': dados_teste['order_id'],
    'delivery_time': previsoes
})

# Salvar o arquivo de submissão
caminho_para_salvar = 'C:/Users/paulo/Downloads/Mac2166/submissao.csv'
submissao.to_csv(caminho_para_salvar, index=False)
print("Arquivo de submissão salvo como 'submissao.csv'")

Mean Absolute Error: 0.10440089829964709
Arquivo de submissão salvo como 'submissao.csv'
